In [3]:
import re
import string
import mlflow
import numpy as np
import pandas as pd
import mlflow.sklearn

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [20]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
270,"Long, boring, blasphemous. Never have I been s...",negative
563,I really enjoyed this movie and it was a littl...,positive
486,Guy Richie's third proper film (not counting t...,negative
994,BROADCAST NEWS opens with a series of brief vi...,positive
321,I liked SOLINO very much. It is a very heart-r...,positive


In [21]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [22]:
df = normalize_text(df)
df.head()

,review,sentiment
270,long boring blasphemous never glad see ending ...,negative
563,really enjoyed movie little difficult brother ...,positive
486,guy richie s third proper film not counting go...,negative
994,broadcast news open series brief vignette clev...,positive
321,liked solino much heart rending story italian ...,positive


In [23]:
df['sentiment'].value_counts()

sentiment
negative    267
positive    233
Name: count, dtype: int64

In [24]:
x = df['sentiment'].isin(['positive', 'negative'])
df = df[x]
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
270,long boring blasphemous never glad see ending ...,0
563,really enjoyed movie little difficult brother ...,1
486,guy richie s third proper film not counting go...,0
994,broadcast news open series brief vignette clev...,1
321,liked solino much heart rending story italian ...,1


In [25]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [61]:
max_features = 100
test_size = 0.20

In [62]:
vectorizer = CountVectorizer(max_features=max_features)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

In [64]:
import dagshub
mlflow.set_tracking_uri('https://dagshub.com/Gavis33/project2-MLOps.mlflow')
dagshub.init(repo_owner='Gavis33', repo_name='project2-MLOps', mlflow=True)

mlflow.set_experiment("Logistic Regression Baseline")

with mlflow.start_run():

    mlflow.sklearn.autolog()
    mlflow.log_param("vectorizer", "Bag of Words")
    mlflow.log_param("num_features",max_features )
    mlflow.log_param("test_size", test_size)

    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    mlflow.log_metric('accuracy', accuracy)
    mlflow.log_metric('precision', precision)
    mlflow.log_metric('recall', recall)
    mlflow.log_metric('f1', f1)
    mlflow.sklearn.log_model(model, 'model')
    mlflow.end_run()

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Initialized MLflow to track repo "Gavis33/project2-MLOps"

Repository Gavis33/project2-MLOps initialized!

2025/05/13 17:05:37 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'
2025/05/13 17:06:03 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run worried-worm-393 at: https://dagshub.com/Gavis33/project2-MLOps.mlflow/#/experiments/0/runs/0d13ed60ef694b4dbdffe584cd19eb34
🧪 View experiment at: https://dagshub.com/Gavis33/project2-MLOps.mlflow/#/experiments/0
Accuracy: 0.62
Precision: 0.6428571428571429
Recall: 0.54
F1 Score: 0.5869565217391305
